In [47]:
import pickle
import os
import wave
import numpy as np
from numpy import array
from shutil import copyfile

import warnings
warnings.simplefilter("ignore", DeprecationWarning)

# глобальные переменные
delta = 50
reduceCount = 5

types = {
    1: np.int8,
    2: np.int16,
    4: np.int32
}

In [48]:
# загрузка сохраненной модели
filename = 'finalized_model.sav'
mlp = pickle.load(open(filename, 'rb'))

In [49]:
def readFromSamplesTxt(filename):
    text_file = open(filename, "r")
    result = text_file.read().split('\n')[:-1]
    return list(map(lambda x: float(x), result))

In [50]:
def convertSamplesInArray(directory):
    files = os.listdir(directory)
    samples = filter(lambda x: x.endswith('.txt'), files)
    output = []

    for sample in samples:
        output.append(readFromSamplesTxt(directory + "/" + sample))
    return output

In [51]:
# def convertSoundsInSamples(directory):
#     files = os.listdir(directory)
#     sounds = filter(lambda x: x.endswith('.wav'), files)

#     for sound in sounds:
#         wav = wave.open(directory + "/" + sound, mode="r")
#         (nchannels, sampwidth, framerate, nframes, comptype, compname) = wav.getparams()

#         content = wav.readframes(nframes)
#         samples = np.fromstring(content, dtype=types[sampwidth])
#         cuttedSamples = cutSamples(samples)
#         filename = sound[:-3]
#         printSamplesInFile(cuttedSamples, directory + "/samples/" + filename + "txt")

In [52]:
# Свертка (сжимает массив сэмплов, усредняя значения)
def cutSamples(samples):
    i = 0
    values = []
    while i + delta < samples.size:
        samplesSlice = samples[i:i+delta]
        samplesSlice = list(map(lambda x: abs(x), samplesSlice))
        samplesSlice.sort()
        samplesSlice = samplesSlice[reduceCount : delta - reduceCount]
        value = sum(samplesSlice) / len(samplesSlice)
        values.append(value)
        i = i + delta
    
    return values

In [53]:
def printSamplesInFile(samples, filename):
    output = open(filename, "w")
    for sample in samples:
        string ='{:.2f}'.format(sample)
        output.write(string + '\n')
    output.close()

In [54]:
# # переводим звуки в текстовые файлы с данными
# convertSoundsInSamples("results/test")

In [55]:
directory = "results/test"
files = os.listdir(directory)
sounds = filter(lambda x: x.endswith('.wav'), files)

for sound in sounds:
    wav = wave.open(directory + "/" + sound, mode="r")
    (nchannels, sampwidth, framerate, nframes, comptype, compname) = wav.getparams()

    content = wav.readframes(nframes)
    samples = np.fromstring(content, dtype=types[sampwidth])
    cuttedSamples = cutSamples(samples)
    
    checked_data = array(cuttedSamples).reshape(1, -1)
    isCough = mlp.predict(checked_data)
    
    source = directory + "/" + sound
    if isCough:
        copyfile(source, directory + "/positive/" + sound)
    else:
        copyfile(source, directory + "/negative/" + sound)